# IPLBot-RASA

# Como construir e fazer deploy de um Chatbot utilizando Rasa (utilizando estudo de caso IPL)

Entendendo um pouco da estrutura do nosso chatbot

![image1.png](../images/image1.png)

Vamos detalhar essa arquitetura (continue consultando a imagem para entender isso):

    * Assim que Rasa recebe uma mensagem do usuário final, ele tenta prever ou extrair a "intenção" e as "entidades" presentes na mensagem. Esta parte é tratada por Rasa NLU
    * Depois que a intenção do usuário é identificada, o Rasa Stack executa uma ação chamada action_match_news para obter as atualizações da última correspondência de IPL
    * Rasa então tenta prever o que deve fazer a seguir. Esta decisão é tomada considerando vários fatores e é tratada pelo Rasa Core
    * No nosso exemplo, Rasa está mostrando o resultado da correspondência mais recente para o usuário. Também previu a próxima ação que nosso modelo deve executar - para verificar com o usuário se o chatbot conseguiu resolver sua consulta

# Configurando o IPL Chatbot

Practice Version – use esta versão ao ler este artigo. Isso ajudará você a entender como o código funciona

Então, vá em frente e clone o projeto 'Practice Version' do GitHub:

```
git clone https://github.com/ClaudioLSilva/IPLBot-chatbot-with-RASA.git && cd IPLBot-chatbot-with-RASA
```

E entre (cd) na practice_version:

```
cd practice_version
```

Uma observação rápida sobre algumas coisas que você deve estar ciente antes de prosseguir:

- Atualmente, o Rasa suporta apenas a versão Python <= 3.6. Se você possui uma versão superior do Python, pode configurar um novo ambiente no conda usando o seguinte comando:

```
conda create -n rasa python=3.6
conda activate rasa
```

- Você precisará de um editor de texto para trabalhar com os vários arquivos do nosso projeto. O meu favorito é o Sublime Text, que você pode baixar [aqui](https://www.sublimetext.com/)

# Instalando o Rasa e suas dependências

Você pode usar o código abaixo para instalar todas as dependências do Rasa Stack:

```
pip install -r requirements.txt
```

Esta etapa pode levar alguns minutos, pois existem alguns arquivos para instalar. Você também precisará instalar um modelo de idioma spaCy em inglês:

```
python -m spacy download en
```

Mãos à obra!

# Extraindo a intenção do usuário de uma mensagem

A primeira coisa que queremos fazer é descobrir a intenção do usuário. O que ele ou ela deseja realizar? Vamos utilizar o Rasa e criar um modelo de NLU para identificar a intenção do usuário e suas entidades relacionadas.

Examine a pasta practice_version que você baixou anteriormente:

![image2.png](../images/image2.png)

Os dois arquivos que usaremos estão destacados acima.

**data/nlu_data.md** - Esse é o arquivo no qual você salvará seus dados de treinamento para extrair a intenção do usuário. Já existem alguns dados no arquivo:

![image3.png](../images/image3.png)

Como você pode ver, o formato dos dados de treinamento para 'intenção' é bastante simples em Rasa. Você só tem de:

* Começar a linha com "## intent: intent_name"
* Fornecer todos os exemplos nas seguintes linhas

Vamos escrever alguns exemplos de intenções em Python para o cenário em que o usuário deseja obter atualizações de IPL:

```
## intent:current_matches
- which cricket match is happening right now 
- which ipl match is next 
- which teams are playing next in ipl 
- tell me some ipl news 
- i want ipl updates 
- what are the latest match updates 
- who won the last ipl match 
- how is ipl going 
- what was the result of the last match 
- when is the next match
```

Você pode incluir quantos exemplos desejar para cada intenção. De fato, certifique-se de incluir gírias e formas curtas que você usa ao enviar mensagens de texto. A idéia é fazer o chatbot entender a maneira como digitamos o texto. Sinta-se à vontade para consultar a versão completa, em que dei muitos exemplos para cada tipo de intenção.

* **nlu_config.yml** - Este arquivo permite criar um pipeline de processamento de texto em Rasa. Felizmente para nós, o Rasa vem com duas configurações padrão com base na quantidade de dados de treinamento que temos:
 * Pipeline "spacy_sklearn" se você tiver menos de 1000 exemplos de treinamento
 * "Tensorflow_embedding" se você tiver uma grande quantidade de dados

Vamos escolher o primeiro como ele se encaixa no nosso exemplo:

![image4.png](../images/image4.png)

Porém, para ampliarmos o uso e a capacidade mais ampliada de interpretação, usaremos a seguinte, configuração:

nlu_config.yml
--------------
```
language: "en"

# pipeline: "spacy_sklearn"  <comentamos esta linha, pois abaixo as mesmas configurações estão embutidas>
pipeline:
  - name: "nlp_spacy"
  - name: "tokenizer_spacy"
  - name: "intent_featurizer_spacy"
  - name: "intent_classifier_sklearn"
  - name: "ner_crf"
  - name: "ner_synonyms"
```

# Treinando o classificador NLU

Se você chegou até aqui, já fez a maior parte do trabalho para o modelo de extração de intenção. Vamos treiná-lo e vê-lo em ação!

Você pode treinar o classificador simplesmente seguindo o comando abaixo:

```
make train-nlu
```
Usando o Windows? Você pode executar o seguinte código python:

```
python -m rasa_nlu.train -c nlu_config.yml --data data/nlu_data.md -o models --fixed_model_name nlu --project current --verbose
```

# Prever a intenção

Vamos testar o desempenho do nosso modelo, fornecendo um texto de exemplo no qual ele não foi treinado para extrair intenções. Você pode abrir um shell iPython / Python e siga as seguintes etapas:
```
>>> from rasa_nlu.model import Interpreter
>>> nlu_model = Interpreter.load('./models/current/nlu')
>>> nlu_model.parse('what is happening in the cricket world these days?')
```

Aqui está a aparência da saída:
```    
{
	'intent': {
	'name': 'current_matches',
	'confidence': 0.6657130383645372
	},
	'entities': [],
	'intent_ranking': [{
	'name': 'current_matches',
	'confidence': 0.6657130383645372
	}, {
	'name': 'goodbye',
	'confidence': 0.11589267701634796
	}, {
	'name': 'affirm',
	'confidence': 0.06372593552204527
	}, {
	'name': 'thanks',
	'confidence': 0.0623182164883462
	}, {
	'name': 'deny',
	'confidence': 0.05938353524001583
	}, {
	'name': 'greet',
	'confidence': 0.03296659736870796
	}],
	'text': 'what is happening in the cricket world these days?'
	}
```

O nosso modelo NLU não apenas tem um bom desempenho na extração de intenções, mas também classifica as outras intenções com base em seus índices de confiança. Este é um pequeno recurso bacana que pode ser realmente útil quando o classificador é confundido entre várias intenções.

# Fazendo conversas interativas

Um dos aspectos mais importantes de um aplicativo de chatbot é a capacidade de ser interativo. Lembre-se de um chatbot que você já usou antes. Nosso interesse naturalmente desperta se o chatbot é capaz de manter uma conversa, certo?

Espera-se que o chatbot extraia todas as informações necessárias para executar uma tarefa específica usando as conversas de vaivém com o usuário final.

# Projetando o fluxo conversacional

Reserve um momento para pensar na conversa mais simples que nosso chatbot pode ter com um usuário. Qual seria o fluxo dessa conversa? Vamos escrever na forma de uma história!

```
Me: Hi

Iplbot: Hey! How may I help you?

Me: What was the result of the last match?

Iplbot: Here are some IPL quick info:
1.The match between Rajasthan Royals and Delhi Capitals was recently held and Delhi Capitals won.
2.The next match is Warriors vs Titans on 22 April 2019

Iplbot: Did that help you?

Me: yes, thank you!

Iplbot: Glad that I could help! :)
```

Vamos ver como podemos ensinar uma conversa simples como essa para Rasa:

```
## news path 1
* greet
  - utter_greet
* current_matches
  - action_match_news
  - utter_did_that_help
* affirm or thanks
  - utter_gratitude
* goodbye
  - utter_goodbye
```

O formato geral é:

```
## news path 1           <--- nome da história para fins de depuração
* greet                  <--- intenção detectada do usuário
  - utter_greet          <--- que ação o bot deve executar
* current_matches        <--- a seguinte intenção na conversa
```

Isso é chamado de **caminho da história do usuário**. Forneci algumas histórias no arquivo **data/stories.md** para sua referência. Esses são os **dados de treinamento do Rasa Core**.

O modo como funciona é:

 * Dê alguns exemplos de caminhos de história de amostra que o usuário deve seguir
 * O Rasa Core os combina aleatoriamente para criar caminhos de usuário mais complexos
 * Em seguida, constrói um modelo probabilístico a partir disso. Este modelo é usado para prever a próxima ação que Rasa deve executar

Confira o arquivo **data/stories.md** na versão completa do projeto para mais exemplos. Enquanto isso, aqui está uma boa visualização dos caminhos básicos da história gerados por Rasa para o nosso chatbot IPL:

![image5.png](../images/image5.png)

A ilustração acima pode parecer complicada, mas está simplesmente listando várias histórias possíveis de usuários que eu ensinei a Rasa. Aqui estão algumas coisas a serem observadas no gráfico acima:

* Exceto pelas caixas INICIAR e FIM, todas as caixas coloridas indicam a intenção do usuário
* Todas as caixas brancas são ações que o chatbot executa
* As setas indicam o fluxo da conversa
* **action_match_news** é onde atingimos o CricAPI para obter informações do IPL

Escreva o seguinte no seu arquivo stories.md:

```
## news path 1
* greet
  - utter_greet
* current_matches
  - action_match_news
  - utter_did_that_help
* affirm or thanks
  - utter_gratitude
* goodbye
  - utter_goodbye

## news path 2
* current_matches
  - action_match_news
  - utter_did_that_help
* affirm or thanks
  - utter_gratitude
* goodbye
  - utter_goodbye

## news path 3
* greet
  - utter_greet
* current_matches
  - action_match_news
  - utter_did_that_help
* deny
  - utter_ask_again
* current_matches
  - action_match_news
  - utter_did_that_help
* affirm or thanks
  - utter_gratitude
* goodbye
  - utter_goodbye

## greet path
* greet
  - utter_greet

## goodbye path
* goodbye
  - utter_goodbye
```

Agora, gere um gráfico semelhante para suas histórias usando o seguinte comando:

```
python -m rasa_core.visualize -d domain.yml -s data/stories.md -o graph.html
```

Porém, para facilitar, embuti no makefile este comando. Então você pode executá-lo mais facilmente assim:
```
make graph
```

**Isso é muito útil ao depurar** o fluxo conversacional do chatbot.

# Definindo o domínio

Agora, abra o arquivo **domain.yml**. Você estará familiarizado com a maioria dos recursos mencionados aqui:

![image6.png](../images/image6.png)

O domínio é o **mundo do seu chatbot**. Ele contém tudo o que o chatbot deve saber, incluindo:

* Todas as ações que é capaz de realizar
* As intenções que ele deve entender
* O modelo de todas as declarações que ele deve informar ao usuário e muito mais

# Definindo diretivas (Policies)

O Rasa Core gera os dados de treinamento para a parte conversacional usando as histórias que fornecemos. Também permite definir um conjunto de políticas a serem usadas ao decidir a próxima ação do chatbot. Essas políticas são definidas no arquivo **policies.yml**.

Então, abra esse arquivo e copie o seguinte código:

```
policies:
  - name: KerasPolicy
    epochs: 100
    max_history: 5
  - name: FallbackPolicy
    fallback_action_name: 'action_default_fallback'
  - name: MemoizationPolicy
    max_history: 5
```

Aqui estão algumas coisas a serem observadas sobre as políticas acima (extraídas das políticas do Rasa Core aqui):

* **O KerasPolicy** usa uma rede neural implementada no Keras para selecionar a próxima ação. A arquitetura padrão é baseada em um modelo LSTM (Long Short Term Term Memory)
* **MemoizationPolicy** memoriza as conversas nos seus dados de treinamento. Ele prevê a próxima ação com confiança 1.0, se essa conversa exata existir nos dados de treinamento, caso contrário, ela prevê 'Nenhuma' com confiança 0.0
* **FallbackPolicy** chama uma ação de fallback se o reconhecimento de intenção tiver confiança abaixo de nlu_threshold ou se nenhuma das políticas de diálogo prever uma ação com confiança maior que core_threshold
* Um hiperparâmetro importante para as políticas do Rasa Core é o **max_history**. Isso controla quanto histórico de diálogo o modelo examina para decidir qual ação tomar em seguida

# Treinando o modelo de conversação

Você pode treinar o modelo usando o seguinte comando:
```
make train-core
```

Ou, se você estiver no Windows, poderá usar o comando Python completo:
```
python -m rasa_core.train -d domain.yml -s data/stories.md -o models/current/dialogue -c policies.yml
```

# Conversando com seu chatbot IPL

Antes de prosseguirmos, vamos conversar com nosso chatbot e ver como ele funciona. Abra um novo terminal e digite o seguinte comando:
```
make cmdline
```

Uma vez carregado, tente conversar com seu chatbot. Você pode começar dizendo "Oi". O vídeo a seguir mostra minha interação com o chatbot:

[![cmdline3.png](../videos/cmdline3.png)](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/04/cmdline3.mp4?_=2)
"Veja este video" (clique com o botao direito do mouse abrindo nova aba)

Recebi uma mensagem de erro ao tentar obter atualizações de IPL:

Foi encontrada uma exceção ao executar a ação 'action_match_news'. O bot continuará, mas os eventos de ações serão perdidos. Certifique-se de corrigir a exceção no seu código personalizado.

O chatbot entendeu minha intenção de receber notícias sobre o IPL. Então, o que deu errado? É simples - ainda não escrevemos o código de back-end para isso! Então, vamos construir o back-end a seguir.

# Obtendo dados IPL usando o CricAPI

Usaremos o CricAPI para obter notícias relacionadas ao IPL. É gratuito para 100 solicitações por dia, o que (espero) é mais do que suficiente para saciar a paixão louca por críquete que você tem.

Você precisa primeiro se inscrever no site para obter acesso à API deles:

https://www.cricapi.com/

Você poderá ver sua chave de API depois de fazer login:

![cricapi.png](../images/cricapi.png)

Salve essa chave, pois será realmente importante para o nosso chatbot. Em seguida, abra o arquivo **actions.py** e atualize-o com o seguinte código:

## actions.py
```
. (mantenha o código acima, e acrescente ou substitua o que for necessário)
.

API_URL = "https://cricapi.com/api/"
API_KEY = "<coloque sua chave api aqui>"

class ApiAction(Action):
	def name(self):
		return "action_match_news"

	def run(self, dispatcher, tracker, domain):
		res = requests.get(API_URL + "matches" + "?apikey=" + API_KEY)
		if res.status_code == 200:
			data = res.json()["matches"]
			recent_match = data[0]
			upcoming_match = data[1]
			upcoming_match["date"] = datetime.strptime(upcoming_match["date"], "%Y-%m-%dT%H:%M:%S.%fZ")
			next_date = upcoming_match["date"].strftime("%d %B %Y")

			out_message = "Here some IPL quick info:\n1.The match between {} and {} was recently held and {} won.".format(recent_match["team-1"], recent_match["team-2"], recent_match["toss_winner_team"])

			dispatcher.utter_message(out_message)

			out_message = "2.The next match is {} vs {} on {}".format(upcoming_match["team-1"], upcoming_match["team-2"], next_date)

			dispatcher.utter_message(out_message)

		return []
```

Preencha o **API_KEY** com o que você recebeu do CricAPI e você deve estar pronto. Agora, você pode tentar novamente falar com seu chatbot. Desta vez, esteja preparado para se surpreender.

Abra um novo terminal e inicie seu servidor de ação:

```
make action-server
```

Isso ativará o servidor que está sendo executado no arquivo **actions.py** e estará funcionando em segundo plano para nós. Agora, reinicie o chatbot na linha de comando:

```
make cmdline
```

E desta vez, ele deve fornecer algumas notícias sobre IPL quando solicitado. Isso não é incrível? Já criamos um chatbot completo sem executar etapas complexas!

# Trazendo o Chatbot à vida (Integrando Rasa e Slack)

Então, temos o chatbot pronto. É hora de implantá-lo e integrá-lo ao Slack, como prometi no início deste artigo. Felizmente para nós, o Rasa lida com 90% da parte da implantação por conta própria.

Nota: Você precisa ter um espaço de trabalho no Slack antes de continuar. Se você não possui um, pode [consultar isto](https://get.slack.help/hc/en-us/articles/206845317-Create-a-Slack-workspace).

# Criando um aplicativo Slack

Agora que temos um espaço de trabalho para experimentar, precisamos de um aplicativo para anexar seu bot. Crie o aplicativo no link abaixo:

https://api.slack.com/apps

(1). Clique em "Criar aplicativo", atribua um nome ao aplicativo e selecione seu espaço de trabalho:

![slack1.png](../images/slack1.png)

Isso o redirecionará para o painel do aplicativo. A partir daí, você pode selecionar a opção **"Bots"**:

![slack2.png](../images/slack2.png)

(2). Clique em "**Adicionar um usuário de bot**" -> Dê um nome ao seu bot. No meu caso, chamei de "**iplbot**". Agora, precisamos adicioná-lo ao nosso espaço de trabalho para que possamos conversar com ele! Volte ao painel do aplicativo acima e role para baixo para encontrar a opção "**Instalar aplicativo no espaço de trabalho**":

![slack3.png](../images/slack3.png)

Depois de fazer isso, o Slack solicitará que você "**autorize**" o aplicativo. Vá em frente e aceite a autorização.

(3). Antes de podermos conectar qualquer programa externo ao nosso bot do Slack, precisamos ter um “**token de autenticação**” que precisamos fornecer ao tentar conectar-se a ele. Volte para o "**painel de aplicativos**" e selecione a opção "**OAuth e permissões**":

![slack4.png](../images/slack4.png)

(4). Isso abrirá as configurações de permissões do aplicativo. Selecione o "**Token de acesso OAuth de usuário bot**" e salve-o (ocultei-o por motivos de segurança). Este token é fundamental para se conectar ao nosso chatbot.

![slack5.png](../images/slack5.png)

# Instalando o Ngrok

Nosso trabalho ainda não acabou. Precisamos de outra ferramenta útil para implantar nosso chatbot no Slack. Isso é [ngrok](https://ngrok.com/) e você pode usar o seguinte link para fazer o download:

https://ngrok.com/download

Agora estamos a um passo de implantar nosso próprio chatbot! Tempos emocionantes nos aguardam na próxima seção.

# Pushing(empurrando) o Chatbot para o Slack

Precisamos de apenas **cinco comandos** para fazer isso, pois Rasa cuida de tudo o mais nos bastidores.

Abra seu arquivo **slack_credentials.yml** e cole o "**Token de acesso OAuth do usuário bot**" no lugar do token Slack:

![slack6.png](../images/slack6.png)

* Vá para um novo terminal e startar o **action-server**:
```
make action-server
```

Você verá que o servidor está sendo executado na **porta 5055**, então vamos usar o *ngrok* nessa porta. Abra outro terminal e digite o seguinte:

```
ngrok http 5055
```

Porém, também acrescentei este comando no Makefile. Então basta usar o comando:
```
make ngrok-slack
```

Isso fornecerá uma saída como a imagem abaixo:

![ngrok1.png](../images/ngrok1.png)

O link destacado é o link da Internet conectado à porta 5055 do seu computador. É o que o ngrok faz - permite que os programas locais do seu computador sejam expostos na Internet. De certa forma, este é um atalho para usar um serviço de nuvem para implantar seu aplicativo.

* Abra seu arquivo **endpoints.yml** e substitua o "http://localhost:5055/webhook" pela URL acima, como este:
```
your_ngrok_url/webhook  
```
obs: # não precisa mencionar a porta, está inclusa no ngrok url

![endpoints.yml.png](../images/endpoints.yml.png)

Faça deploy do chatbot Rasa usando o seguinte comando:
```
python -m rasa_core.run -d models/current/dialogue -u models/current/nlu --port 5002 --connector slack --credentials slack_credentials.yml --endpoints endpoints.yml
```
Mas, para faciltar, também acrescentei no Makefile. Então use o comando:
```
make deploy-rasa-chatbot
```

Você terá uma resposta com esta:

![deploy-rasa-chatbot.png](../images/deploy-rasa-chatbot.png)

Observe que o servidor Rasa Core está sendo executado na **porta 5002**.

* Agora, implante a porta 5002 na Internet:
```
ngrok http 5002
```
* Vá para o painel do aplicativo no Slack, clique em **Assinatura de eventos (Events Subscription)** e, em seguida, no botão "**Ativar inscrições de evento (Enable Event Subscriptions)**". Cole a URL ngrok do seu Rasa Core Server neste formato no campo **Solicitar URL (Request URL)**:

![request_url.png](../images/request_url.png)

No URL acima, substitua a parte ngrok pelo seu URL ngrok:
```
your_ngrok_url/webhooks/slack/webhook
```
* Sob **Inscrever-se em Eventos de Bot**, clique no botão **Adicionar Evento de Usuário de Bot**. Ele irá revelar um campo de texto e uma lista de eventos. Você pode inserir termos nesse campo para pesquisar os eventos aos quais você deseja que seu bot responda. Aqui está uma lista de eventos que sugiro adicionar:

![subscribe-bot-events.png](../images/subscribe-bot-events.png)

Depois de adicionar os eventos, clique no botão Salvar alterações na parte inferior da tela.

Agora você pode simplesmente atualizar sua página do Slack e começar a conversar imediatamente com seu bot! Aqui está uma conversa com meu chatbot:

[![chat-bot-talks-on-slack.png](../videos/chat-bot-talks-on-slack.png)](https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2019/04/iplbot21.mp4?_=3)
"Veja este video" (clique com o botao direito do mouse abrindo nova aba)

# Para onde você deve ir daqui?

Você encontrará os links abaixo úteis se estiver procurando por desafios semelhantes. Criei um chatbot do tipo Zomato para problemas de **Pesquisa de restaurante** usando os modelos Rasa Core e Rasa NLU. Vou ensinar isso com muito mais detalhes em nosso curso sobre Processamento de linguagem natural.

Os links para o curso estão abaixo para sua referência:

* [Certified Course: Natural Language Processing (NLP) using Python](https://courses.analyticsvidhya.com/courses/natural-language-processing-nlp/?utm_source=blog&utm_medium=learn-build-chatbot-rasa-nlp-ipl)
* [Certified Program: NLP for Beginners](https://courses.analyticsvidhya.com/bundles/nlp-combo?utm_source=blog&utm_medium=learn-build-chatbot-rasa-nlp-ipl)
* [The Ultimate AI & ML BlackBelt Program](https://courses.analyticsvidhya.com/bundles/ai-blackbelt-beginner-to-master?utm_source=blog&utm_medium=learn-build-chatbot-rasa-nlp-ipl)

# Notas finais

Eu adoraria ver diferentes abordagens e técnicas da nossa comunidade. Tente usar diferentes pipelines no Rasa Core, explore mais políticas, ajuste esses modelos, verifique quais outros recursos o CricAPI oferece, etc. Há tantas coisas que você pode tentar! Não se detenha aqui - continue e experimente.

Sinta-se à vontade para discutir e fornecer seus comentários na seção de comentários abaixo. O código completo do meu projeto está disponível [aqui](https://github.com/mohdsanadzakirizvi/iplbot).

Você também deve verificar estes dois artigos sobre a criação de chatbots:

* [A Guide to Building an Intelligent Chatbot for Slack using DialogFlow API](https://www.analyticsvidhya.com/blog/2018/03/how-to-build-an-intelligent-chatbot-for-slack-using-dialogflow-api/?utm_source=blog&utm_medium=learn-build-chatbot-rasa-nlp-ipl)
* [Building a FAQ Chatbot in Python, The Future of Information Searching](https://www.analyticsvidhya.com/blog/2018/01/faq-chatbots-the-future-of-information-searching/?utm_source=blog&utm_medium=learn-build-chatbot-rasa-nlp-ipl)

Você também pode ler este artigo no APP Android do Analytics Vidhya Adquira no Google Play ![google-play.png](../images/google-play.png)

>>>> PS: **Este manual foi traduzido e modificado/acrescentado com base [neste documento](https://www.analyticsvidhya.com/blog/2019/04/learn-build-chatbot-rasa-nlp-ipl/), cujo autor é [Mohd Sanad Zaki Rizvi, April 29, 2019](https://www.analyticsvidhya.com/blog/author/mohdsanadzakirizvigmail-com/). Qualquer modificação/acréscimo será em benefício de nossa comunidade Smart - Obrigado aà todos**